In [1]:
import openmc
import openmc.deplete
import numpy as np
import pandas as pd
from uncertainties import ufloat
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from collections.abc import Iterable
from itertools import chain

model = openmc.model.Model()

In [2]:
plot_openmc_geometry = 0

def pitch_to_edge(pitch):
    return 0.5 * pitch / np.sin(np.pi / 3.0)


def fill_rings(n_rings, univ):

    if not isinstance(univ, Iterable):
        univs = [univ] * n_rings
    else:
        univs = univ

    univs_out = []

    for ring in range(n_rings):
        if ring == 0:
            univs_out.append([univs[0]])
        else:
            if len(univs) != n_rings:
                raise RuntimeError("Incorrect number of universes.")
            univs_out.append(6*ring*[univs[ring]])

    return univs_out[::-1]

#--------------------------------------------------------------
#                            Materials
#--------------------------------------------------------------
inner_fuel_mat = openmc.Material(name='inner fuel mat')
inner_fuel_mat.temperature = 900
inner_fuel_mat.set_density('g/cc', 11.3726)
inner_fuel_mat.add_nuclide('U235', 9.9797e-04, 'ao')
inner_fuel_mat.add_nuclide('U236', 7.9499e-05, 'ao')
inner_fuel_mat.add_nuclide('U238', 6.4448e-01, 'ao')
inner_fuel_mat.add_nuclide('Np237', 7.1247e-04, 'ao')
inner_fuel_mat.add_nuclide('Pu238', 3.0546e-04, 'ao')
inner_fuel_mat.add_nuclide('Pu239', 1.1040e-01, 'ao')
inner_fuel_mat.add_nuclide('Pu240', 1.4807e-02, 'ao')
inner_fuel_mat.add_nuclide('Pu241', 1.5970e-03, 'ao')
inner_fuel_mat.add_nuclide('Pu242', 7.0685e-04, 'ao')
inner_fuel_mat.add_nuclide('Am241', 7.3957e-04, 'ao')
inner_fuel_mat.add_nuclide('Am242_m1', 1.9382e-05, 'ao')
inner_fuel_mat.add_nuclide('Am243', 1.5442e-04, 'ao')
inner_fuel_mat.add_nuclide('Cm244', 3.8446e-05, 'ao')
inner_fuel_mat.add_nuclide('Zr90', 1.1740e-01, 'ao')
inner_fuel_mat.add_nuclide('Zr91', 2.5320e-02, 'ao')
inner_fuel_mat.add_nuclide('Zr92', 3.8282e-02, 'ao')
inner_fuel_mat.add_nuclide('Zr94', 3.7968e-02, 'ao')
inner_fuel_mat.add_nuclide('Zr96', 5.9892e-03, 'ao')

outer_fuel_mat = openmc.Material(name='outer fuel mat')
outer_fuel_mat.temperature = 900 
outer_fuel_mat.set_density('g/cc', 11.3726)
outer_fuel_mat.add_nuclide('U235', 9.4779e-04, 'ao')
outer_fuel_mat.add_nuclide('U236', 7.5502e-05, 'ao')
outer_fuel_mat.add_nuclide('U238', 6.1208e-01, 'ao')
outer_fuel_mat.add_nuclide('Np237', 6.7665e-04, 'ao')
outer_fuel_mat.add_nuclide('Pu238', 3.8322e-04, 'ao')
outer_fuel_mat.add_nuclide('Pu239', 1.3850e-01, 'ao')
outer_fuel_mat.add_nuclide('Pu240', 1.8577e-02, 'ao')
outer_fuel_mat.add_nuclide('Pu241', 2.0036e-03, 'ao')
outer_fuel_mat.add_nuclide('Pu242', 8.8679e-04, 'ao')
outer_fuel_mat.add_nuclide('Am241', 7.0239e-04, 'ao')
outer_fuel_mat.add_nuclide('Am242_m1', 1.8408e-05, 'ao')
outer_fuel_mat.add_nuclide('Am243', 1.4665e-04, 'ao')
outer_fuel_mat.add_nuclide('Cm244', 3.6513e-05, 'ao')
outer_fuel_mat.add_nuclide('Zr90', 1.1740e-01, 'ao')
outer_fuel_mat.add_nuclide('Zr91', 2.5320e-02, 'ao')
outer_fuel_mat.add_nuclide('Zr92', 3.8282e-02, 'ao')
outer_fuel_mat.add_nuclide('Zr94', 3.7968e-02, 'ao')
outer_fuel_mat.add_nuclide('Zr96', 5.9892e-03, 'ao')

test_fuel_mat = openmc.Material(name='test fuel mat')
test_fuel_mat.temperature = 900 
test_fuel_mat.set_density('g/cc', 11.3726)
test_fuel_mat.add_nuclide('U235', 9.7169e-04, 'ao')
test_fuel_mat.add_nuclide('U236', 7.7406e-05, 'ao')
test_fuel_mat.add_nuclide('U238', 6.2751e-01, 'ao')
test_fuel_mat.add_nuclide('Np237', 6.9371e-04, 'ao')
test_fuel_mat.add_nuclide('Pu238', 3.4619e-04, 'ao')
test_fuel_mat.add_nuclide('Pu239', 1.2512e-01, 'ao')
test_fuel_mat.add_nuclide('Pu240', 1.6782e-02, 'ao')
test_fuel_mat.add_nuclide('Pu241', 1.8100e-03, 'ao')
test_fuel_mat.add_nuclide('Pu242', 8.0110e-04, 'ao')
test_fuel_mat.add_nuclide('Am241', 7.2010e-04, 'ao')
test_fuel_mat.add_nuclide('Am242_m1', 1.8872e-05, 'ao')
test_fuel_mat.add_nuclide('Am243', 1.5035e-04, 'ao')
test_fuel_mat.add_nuclide('Cm244', 3.7434e-05, 'ao')
test_fuel_mat.add_nuclide('Zr90', 1.1740e-01, 'ao')
test_fuel_mat.add_nuclide('Zr91', 2.5320e-02, 'ao')
test_fuel_mat.add_nuclide('Zr92', 3.8282e-02, 'ao')
test_fuel_mat.add_nuclide('Zr94', 3.7968e-02, 'ao')
test_fuel_mat.add_nuclide('Zr96', 5.9892e-03, 'ao')

sodium = openmc.Material(name='sodium')
sodium.temperature = 700 
sodium.set_density('g/cc', 0.85)
sodium.add_nuclide('Na23', 1.0, 'ao')

ht9 = openmc.Material(name='ht9')
ht9.temperature = 700 
ht9.set_density('g/cc', 7.648)
ht9.add_nuclide('Fe54', 5.0024e-02, 'ao')
ht9.add_nuclide('Fe56', 7.8525e-01, 'ao')
ht9.add_nuclide('Fe57', 1.8135e-02, 'ao')
ht9.add_nuclide('Fe58', 2.4130e-03, 'ao')
ht9.add_nuclide('Ni58', 3.5920e-03, 'ao')
ht9.add_nuclide('Ni60', 1.3840e-03, 'ao')
ht9.add_nuclide('Ni61', 6.0000e-05, 'ao')
ht9.add_nuclide('Ni62', 1.9200e-04, 'ao')
ht9.add_nuclide('Ni64', 4.9000e-05, 'ao')
ht9.add_nuclide('Cr50', 5.5290e-03, 'ao')
ht9.add_nuclide('Cr52', 1.0662e-01, 'ao')
ht9.add_nuclide('Cr53', 1.2090e-02, 'ao')
ht9.add_nuclide('Cr54', 3.0090e-03, 'ao')
ht9.add_nuclide('Mn55', 5.6370e-03, 'ao')
ht9.add_nuclide('Mo92', 8.9300e-04, 'ao')
ht9.add_nuclide('Mo94', 5.5600e-04, 'ao')
ht9.add_nuclide('Mo95', 9.5800e-04, 'ao')
ht9.add_nuclide('Mo96', 1.0030e-03, 'ao')
ht9.add_nuclide('Mo97', 5.7500e-04, 'ao')
ht9.add_nuclide('Mo98', 1.4520e-03, 'ao')
ht9.add_nuclide('Mo100', 5.7900e-04, 'ao')

shield_b4c = openmc.Material(name='shield b4c')
shield_b4c.temperature = 700 
shield_b4c.set_density('g/cc', 2.0412)
shield_b4c.add_nuclide('B10', 1.60e-01, 'ao')
shield_b4c.add_nuclide('B11', 6.40e-01, 'ao')
shield_b4c.add_element('C', 2.00e-01, 'ao')

CRSR_b4c = openmc.Material(name='CRSR_b4c')
CRSR_b4c.temperature = 700 
CRSR_b4c.set_density('g/cc', 2.142)
CRSR_b4c.add_nuclide('B10', 1.60e-01, 'ao')
CRSR_b4c.add_nuclide('B11', 6.40e-01, 'ao')
CRSR_b4c.add_element('C', 2.00e-01, 'ao')

void = openmc.Material(name='void')
void.set_density('atom/b-cm', 1.e-10)
void.add_nuclide('He4', 1, 'ao')

# MATERIAL BLENDS

radial_reflector = openmc.Material.mix_materials([sodium, ht9],
                                                 [0.1573, 0.8427],
                                                 'vo')
radial_reflector.temperature = 700

lower_reflector = openmc.Material.mix_materials([sodium, ht9],
                                                [0.3208, 0.6792],
                                                'vo')
lower_reflector.temperature = 700

shield_mat = openmc.Material.mix_materials([sodium, ht9, shield_b4c],
                                           [0.3041, 0.1730, 0.5229],
                                           'vo')
shield_mat.temperature = 700

upper_naplenum = openmc.Material.mix_materials([sodium, ht9],
                                               [0.7682, 0.2318],
                                               'vo')
upper_naplenum.temperature = 700

upper_gasplenum = openmc.Material.mix_materials([sodium, ht9, void],
                                                [0.3208, 0.2318, 0.4474],
                                                'vo')
upper_gasplenum.temperature = 700

control_empty = openmc.Material.mix_materials([ht9, sodium],
                                              [0.0783, 0.9217],
                                              'vo')
control_empty.temperature = 700

#--------------------------------------------------------------
#                            GEOMETRY
#--------------------------------------------------------------
# core-level surfaces
z0 = openmc.ZPlane(z0=0.0, boundary_type='vacuum')
zLowCore = openmc.ZPlane(z0=60.)
zLowCore_CR = openmc.ZPlane(z0=140.)
zTopDF = openmc.ZPlane(z0=140.)
zTopDF_CR = openmc.ZPlane(z0=260., boundary_type='vacuum')
zTopNaGP = openmc.ZPlane(z0=160.)
zTopGP = openmc.ZPlane(z0=260., boundary_type='vacuum')
test_height = openmc.ZPlane(z0=180.)


assmbly_pitch_val = 14.598
sub_assembly = openmc.model.hexagonal_prism(pitch_to_edge(assmbly_pitch_val), orientation='y')
pin_DF_pitch_val = 0.908
pin_CR_pitch_val = 1.243
pin_CR_pitch = openmc.model.hexagonal_prism(edge_length=pitch_to_edge(pin_CR_pitch_val), orientation='x')
outer_duct_to_duct = openmc.model.hexagonal_prism(edge_length=pitch_to_edge(14.198), orientation='y')
inner_duct_to_duct = openmc.model.hexagonal_prism(edge_length=pitch_to_edge(13.598), orientation='y')

Path = 0.2032*1e2
Rd = 0.00103*1e2
Ro = 0.400
Fuel_RoEq = Ro * np.sqrt(1+(Rd/Ro)**2 * np.sqrt(1+(np.pi*(Ro*2)/Path)**2))
cladding_outer = openmc.ZCylinder(r=Fuel_RoEq)
cladding_inner = openmc.ZCylinder(r=0.348)


Path = 0.2032*1e2
Rd = 0.00133*1e2
Ro = 0.555
CR_RoEq = Ro * np.sqrt(1+(Rd/Ro)**2 * np.sqrt(1+(np.pi*(Ro*2)/Path)**2))
cladding_outer_CR = openmc.ZCylinder(r=CR_RoEq)
cladding_inner_CR = openmc.ZCylinder(r=0.485)

outer_duct_to_duct_CR = openmc.model.hexagonal_prism(pitch_to_edge(12.798), orientation='y')
inner_duct_to_duct_CR = openmc.model.hexagonal_prism(pitch_to_edge(12.198), orientation='y')

# control_rod assembly

CR_pin_inner_univ = openmc.model.pin([cladding_inner_CR, cladding_outer_CR],
                                     [CRSR_b4c, ht9, sodium])
CR_pin = openmc.Cell(fill=CR_pin_inner_univ, region=pin_CR_pitch)

CR_assembly = openmc.HexLattice()
CR_assembly.pitch = (pin_CR_pitch_val,)
CR_assembly.universes = fill_rings(6, CR_pin_inner_univ)
CR_assembly.center = (0.0, 0.0)

e_length = pitch_to_edge(CR_assembly.pitch[0] * 15)
CR_assembly_hex = openmc.model.hexagonal_prism(edge_length=e_length,
                                         boundary_type='vacuum',
                                         orientation='y')


sodium_cell = openmc.Cell(fill=sodium, name='inf_sodium')
sodium_univ = openmc.Universe(cells=[sodium_cell])

# Define the duct regions once and clone them when needed in universe/lattice
assembly_duct = openmc.Cell(fill=ht9, region=+zLowCore & -zTopDF & ~inner_duct_to_duct & outer_duct_to_duct)
assembly_outer = openmc.Cell(fill=sodium, region=+zLowCore & -zTopDF & ~outer_duct_to_duct)

assembly_duct_CR = openmc.Cell(fill=ht9, region=+zLowCore_CR & ~inner_duct_to_duct_CR & outer_duct_to_duct_CR)
assembly_outer_CR = openmc.Cell(fill=sodium, region=+zLowCore_CR & ~outer_duct_to_duct_CR)

lower_reflector = openmc.Cell(fill=lower_reflector, region=+z0 & -zLowCore)
na_plenum = openmc.Cell(fill=upper_naplenum, region=+zTopDF & -zTopNaGP)
gas_plenum = openmc.Cell(fill=upper_gasplenum, region=+zTopNaGP & -zTopGP)


# Control Rod Assembly

# empty region
control_rod = openmc.Cell(name='control_rod', fill=control_empty, region=+z0 & -zLowCore_CR)

control_rod_pin = openmc.model.funcs.pin([cladding_inner_CR, cladding_outer_CR],
                                         [CRSR_b4c, ht9, sodium])

control_rod_lattice = openmc.HexLattice()
control_rod_lattice.pitch = (pin_CR_pitch_val,)
control_rod_lattice.orientation = 'y'
control_rod_lattice.center = (0.0, 0.0)
control_rod_lattice.outer = sodium_univ
control_rod_lattice.universes = fill_rings(9, control_rod_pin)

control_rod_assembly = openmc.Cell(fill=control_rod_lattice, region=+zLowCore_CR & inner_duct_to_duct_CR)
control_rod_duct = assembly_duct_CR.clone(False, False)
control_rot_outer = assembly_outer_CR.clone(False, False)

control_rod_univ = openmc.Universe(cells=[control_rod, control_rod_assembly, control_rod_duct, control_rot_outer])

# Inner Fuel Assembly

inner_fuel_lower_refl = lower_reflector.clone(False, False)

inner_fuel_pin = openmc.model.pin([cladding_inner, cladding_outer],
                                        [inner_fuel_mat, ht9, sodium])

inner_fuel_lattice = openmc.HexLattice()
inner_fuel_lattice.pitch = (pin_DF_pitch_val,)
inner_fuel_lattice.orientation = 'y'
inner_fuel_lattice.center = (0.0, 0.0)
inner_fuel_lattice.outer = sodium_univ

inner_fuel_lattice.universes = fill_rings(9, inner_fuel_pin)

inner_fuel_assembly = openmc.Cell(fill=inner_fuel_lattice, region=+zLowCore & -zTopDF & inner_duct_to_duct)
inner_fuel_duct = assembly_duct.clone(False, False)
inner_fuel_outer = assembly_outer.clone(False, False)

inner_fuel_na_plenum = na_plenum.clone(False, False)
inner_fuel_gas_plenum = gas_plenum.clone(False, False)

inner_fuel_univ = openmc.Universe(cells=[inner_fuel_lower_refl,
                                         inner_fuel_assembly,
                                         inner_fuel_duct,
                                         inner_fuel_outer,
                                         inner_fuel_na_plenum,
                                         inner_fuel_gas_plenum])

# Test Fuel Assembly

test_fuel_lower_refl = lower_reflector.clone(False, False)

test_fuel_pin = openmc.model.pin([cladding_inner, cladding_outer],
                                       [test_fuel_mat, ht9, sodium])

test_fuel_lattice = openmc.HexLattice()
test_fuel_lattice.pitch = (pin_DF_pitch_val,)
test_fuel_lattice.orientation = 'y'
test_fuel_lattice.center = (0.0, 0.0)
test_fuel_lattice.outer = sodium_univ
test_fuel_lattice.universes = fill_rings(9, test_fuel_pin)

test_fuel_assembly = openmc.Cell(fill=test_fuel_lattice, region=+zLowCore & -zTopDF & inner_duct_to_duct)
test_fuel_duct = assembly_duct.clone(False, False)
test_fuel_outer = assembly_outer.clone(False, False)

test_fuel_na_plenum = na_plenum.clone(False, False)

test_fuel_gas_plenum = gas_plenum.clone(False, False)

test_fuel_univ = openmc.Universe(cells=[test_fuel_lower_refl,
                                        test_fuel_assembly,
                                        test_fuel_duct,
                                        test_fuel_outer,
                                        test_fuel_na_plenum,
                                        test_fuel_gas_plenum])

# Outer Fuel Assembly

outer_fuel_lower_refl = lower_reflector.clone(False, False)

outer_fuel_pin = openmc.model.pin([cladding_inner, cladding_outer],
                                       [outer_fuel_mat, ht9, sodium])

outer_fuel_lattice = openmc.HexLattice()
outer_fuel_lattice.pitch = (pin_DF_pitch_val,)
outer_fuel_lattice.orientation = 'y'
outer_fuel_lattice.center = (0.0, 0.0)
outer_fuel_lattice.outer = sodium_univ
outer_fuel_lattice.universes = fill_rings(9, outer_fuel_pin)

outer_fuel_assembly = openmc.Cell(fill=outer_fuel_lattice, region=+zLowCore & -zTopDF & inner_duct_to_duct)
outer_fuel_duct = assembly_duct.clone(False, False)
outer_fuel_outer = assembly_outer.clone(False, False)

outer_fuel_na_plenum = na_plenum.clone(False, False)
outeR_fuel_gas_plenum = gas_plenum.clone(False, False)

outer_fuel_univ = openmc.Universe(cells=[outer_fuel_lower_refl,
                                         outer_fuel_assembly,
                                         outer_fuel_duct,
                                         outer_fuel_outer,
                                         outer_fuel_na_plenum,
                                         outeR_fuel_gas_plenum])

# Homogenized core regions (reflector, shielding, void)

reflector = openmc.Cell(name='reflector', fill=radial_reflector)
reflector_univ = openmc.Universe(cells=[reflector])

shielding = openmc.Cell(name='shielding', fill=shield_b4c)
shielding_univ = openmc.Universe(cells=[shielding])

void_cell = openmc.Cell(name='void')
void_univ = openmc.Universe(cells=[void_cell])

reactor_lattice = openmc.HexLattice(name='core lattice')
reactor_lattice.pitch = (assmbly_pitch_val,)
reactor_lattice.orientation = 'x'
reactor_lattice.center = (0.0, 0.0)
reactor_lattice.outer = void_univ

ring_univs = [control_rod_univ,
              inner_fuel_univ,
              inner_fuel_univ,
              inner_fuel_univ,
              outer_fuel_univ,
              outer_fuel_univ,
              reflector_univ,
              reflector_univ,
              shielding_univ]
reactor_lattice.universes = fill_rings(9, ring_univs)

reactor_lattice.universes[-3][1] = test_fuel_univ
reactor_lattice.universes[-3][3] = control_rod_univ
reactor_lattice.universes[-3][5] = test_fuel_univ
reactor_lattice.universes[-3][7] = control_rod_univ
reactor_lattice.universes[-3][9] = test_fuel_univ
reactor_lattice.universes[-3][11] = control_rod_univ

reactor_lattice.universes[-4][0] = test_fuel_univ
reactor_lattice.universes[-4][3] = reflector_univ
reactor_lattice.universes[-4][6] = test_fuel_univ
reactor_lattice.universes[-4][9] = reflector_univ
reactor_lattice.universes[-4][12] = test_fuel_univ
reactor_lattice.universes[-4][15] = reflector_univ

reactor_lattice.universes[-5][2] = control_rod_univ
reactor_lattice.universes[-5][6] = control_rod_univ
reactor_lattice.universes[-5][10] = control_rod_univ
reactor_lattice.universes[-5][14] = control_rod_univ
reactor_lattice.universes[-5][18] = control_rod_univ
reactor_lattice.universes[-5][22] = control_rod_univ

reactor_lattice.universes[-6][0] = reflector_univ
reactor_lattice.universes[-6][1] = reflector_univ
reactor_lattice.universes[-6][4] = reflector_univ
reactor_lattice.universes[-6][5] = reflector_univ
reactor_lattice.universes[-6][6] = reflector_univ
reactor_lattice.universes[-6][9] = reflector_univ
reactor_lattice.universes[-6][10] = reflector_univ
reactor_lattice.universes[-6][11] = reflector_univ
reactor_lattice.universes[-6][14] = reflector_univ
reactor_lattice.universes[-6][15] = reflector_univ
reactor_lattice.universes[-6][16] = reflector_univ
reactor_lattice.universes[-6][19] = reflector_univ
reactor_lattice.universes[-6][20] = reflector_univ
reactor_lattice.universes[-6][21] = reflector_univ
reactor_lattice.universes[-6][24] = reflector_univ
reactor_lattice.universes[-6][25] = reflector_univ
reactor_lattice.universes[-6][26] = reflector_univ

reactor_lattice.universes[-8][0] = shielding_univ
reactor_lattice.universes[-8][1] = shielding_univ
reactor_lattice.universes[-8][6] = shielding_univ
reactor_lattice.universes[-8][7] = shielding_univ
reactor_lattice.universes[-8][8] = shielding_univ
reactor_lattice.universes[-8][13] = shielding_univ
reactor_lattice.universes[-8][14] = shielding_univ
reactor_lattice.universes[-8][15]= shielding_univ
reactor_lattice.universes[-8][20] = shielding_univ
reactor_lattice.universes[-8][21] = shielding_univ
reactor_lattice.universes[-8][22] = shielding_univ
reactor_lattice.universes[-8][27] = shielding_univ
reactor_lattice.universes[-8][28] = shielding_univ
reactor_lattice.universes[-8][29] = shielding_univ
reactor_lattice.universes[-8][34] = shielding_univ
reactor_lattice.universes[-8][35] = shielding_univ
reactor_lattice.universes[-8][36] = shielding_univ
reactor_lattice.universes[-8][41] = shielding_univ

reactor_lattice.universes[-9][0] = void_univ
reactor_lattice.universes[-9][1] = void_univ
reactor_lattice.universes[-9][7] = void_univ
reactor_lattice.universes[-9][8] = void_univ
reactor_lattice.universes[-9][9] = void_univ
reactor_lattice.universes[-9][15] = void_univ
reactor_lattice.universes[-9][16] = void_univ
reactor_lattice.universes[-9][17] = void_univ
reactor_lattice.universes[-9][23] = void_univ
reactor_lattice.universes[-9][24] = void_univ
reactor_lattice.universes[-9][25] = void_univ
reactor_lattice.universes[-9][31] = void_univ
reactor_lattice.universes[-9][32] = void_univ
reactor_lattice.universes[-9][33] = void_univ
reactor_lattice.universes[-9][39] = void_univ
reactor_lattice.universes[-9][40] = void_univ
reactor_lattice.universes[-9][41] = void_univ
reactor_lattice.universes[-9][47] = void_univ

reactor_hex = openmc.model.hexagonal_prism(edge_length=9 * reactor_lattice.pitch[0],
                                     boundary_type='vacuum',
                                     orientation='x')

# reactor core universe
root_universe = openmc.Universe(cells=[
    openmc.Cell(fill = reactor_lattice, region = +z0 & -zTopGP & reactor_hex)
])

model.geometry = openmc.Geometry(root=root_universe)

#--------------------------------------------------------------
#                      SETTINGS
#--------------------------------------------------------------

settings = openmc.Settings()
settings.particles = int(50_000)
settings.batches = 100
settings.inactive = 50
settings.temperature['method'] = 'interpolation'
settings.output = {
    'tallies' : False, 
    'summary': True
}

bbox = ((-100.0, -100.0, 60.0), (100.0, 100.0, 140.0))
settings.source = openmc.IndependentSource(space=openmc.stats.Box(*bbox))


model.settings = settings

#--------------------------------------------------------------
#                       TALLIES
#--------------------------------------------------------------
tallies = openmc.Tallies()

tally_f1 = openmc.Tally(name='fuel_1_pintally_fuel_mat_1')
tally_f1.filters = [ 
    openmc.EnergyFilter([0, 0.625, 100., 2e7]), 
    openmc.DistribcellFilter(next(iter(inner_fuel_pin.cells.values()))) 
]
tally_f1.scores = ['heating', 'flux']
tallies.append(tally_f1)

tally_f2 = openmc.Tally(name='fuel_2_pintally_fuel_mat_2')
tally_f2.filters = [ 
    openmc.EnergyFilter([0, 0.625, 1e1, 2e7]), 
    openmc.DistribcellFilter(next(iter(outer_fuel_pin.cells.values())))
]
tally_f2.scores = ['heating', 'flux']
tallies.append(tally_f2)

tally_uni = openmc.Tally(name='assembly average score')

universes = model.geometry.get_lattices_by_name(name='core lattice')[0].universes
universes = list(chain.from_iterable(universes))

tally_uni.filters = [ 
    openmc.EnergyFilter([0, 0.625, 1e1, 2e7]),
    openmc.UniverseFilter(universes)
]
tally_uni.scores = ['fission', 'flux', '(n,gamma)', 'absorption']
tallies.append(tally_uni)

model.tallies = tallies

#--------------------------------------------------------------
#                    PLOT
#--------------------------------------------------------------
plots = openmc.Plots()

plot = openmc.Plot()
plot.origin = (0, 0, 100)
plot.width = (120*2, 120*2)
plot.pixels = (1000, 1000)
plot.color_by = 'material'
plot.level = 4

plots.append(plot)

model.plots = plots

if plot_openmc_geometry == 1:
    model.plot_geometry()

# Run OpenMC
model.run(threads=6)

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

PosixPath('/home/ariful/temp/openmc_test/github/statepoint.100.h5')